In [2]:
flx_start = 25181192 - 2000
flx_end = 25188563 + 2000

print(flx_start)
print(flx_end)

25179192
25190563


In [1]:
#imports

import os
from Bio import SeqIO
from shutil import copyfile

In [2]:
# important file paths and variables

plasmid_path = '/XXX/plasmids/addgene-plasmid-107312-sequence-237633.gbk'
original_fasta = '/XXX/original_files_for_genome_build/Mus_musculus.GRCm39.dna.primary_assembly.fa'
mod_fasta_path = 'XXX/mcherry_nr1_genome_expand/fasta.fa'
original_gtf = '/XXX/original_files_for_genome_build/Mus_musculus.GRCm39.104.gtf'
mod_gtf_path = '/XXX/mcherry_nr1_genome_expand/genes.gtf'
filtered_gtf_path = 'XXX/mcherry_nr1_genome_expand/genes.filter.gtf'
bed_path = 'XXX/opt_items/mcherry_nr1_genome_expand/genes.filter.bed'
genome_name = 'modified_genome'

flx_start = 25181192 - 2000
flx_end = 25188563 + 2000
flx_chr = '2'
buffer = 0

In [3]:
### amend the fasta file

# get the plasmid
from Bio import SeqIO
gb_file = plasmid_path
for gb_record in SeqIO.parse(open(gb_file,"r"), "genbank") :
    # now do something with the record
    print("Name %s, %i features" % (gb_record.name, len(gb_record.features)))
    print(repr(gb_record.seq))    
mcherry_raw_seq = str(getattr(gb_record,'seq'))

#use the original fasta to build new headers
with open (original_fasta, "r") as myfile:
    genome_list=myfile.readlines()
    
matched_items = [item for item in genome_list if ">" in item] #list comprehension to find the headers
template_header = matched_items[20]
virus_header = template_header.replace('X', 'V')
virus_header = virus_header.replace('171031299', str(len(mcherry_raw_seq)))

### break the plasmid string into 61 character increments for each line
def chunkstring(string, length):
    return (string[0+i:length+i] for i in range(0, len(string), length))
broken_dna = list(chunkstring(mcherry_raw_seq, 61))  ### this is where we've stored our list
broken_dna = ['{0}\n'.format(element) for element in broken_dna]  #add new line characters

#now place the plasmid sequences/lines sequence after the header
fasta_input_list = []
fasta_input_list.extend(broken_dna) 
fasta_input_list.insert(0, virus_header)

#copy the original file
copyfile(original_fasta, mod_fasta_path)

#write to that file
with open(mod_fasta_path, "a") as myfile:
    for item in fasta_input_list:
        myfile.write(item)

Name Exported, 28 features
Seq('CCTGCAGGCAGCTGCGCGCTCGCTCGCTCACTGAGGCCGCCCGGGCGTCGGGCG...TGT', IUPACAmbiguousDNA())


In [4]:
# alter the Gtf file

#filter out the features that overlap with the floxed region

i = 0
with open(original_gtf, "r") as file_input:
    with open(mod_gtf_path, "w") as output:
        for line in file_input:
            if i < 5:
                i = i + 1
                output.write(line)
            else:
                current_features = line.split("\t")
                feature_start = int(current_features[3])
                feature_end = int(current_features[4])
                feature_chr = current_features[0]
                feature_strand = current_features[6]
                if (feature_chr != flx_chr):
                    output.write(line)
                elif (feature_chr == flx_chr) and (feature_start <= flx_start) and (feature_end >= flx_end):
                    continue
                elif (feature_chr == flx_chr) and (feature_start >= flx_start) and (feature_start <= flx_end):
                    continue
                elif (feature_chr == flx_chr) and (feature_end >= flx_start) and (feature_end <= flx_end):
                    continue
                elif (feature_chr == flx_chr) and (feature_start <= (flx_start-buffer)):
                    output.write(line)
                    continue
                elif (feature_chr == flx_chr) and (feature_end >= (flx_end+buffer)):
                    output.write(line)
                    continue
                elif (feature_chr == flx_chr) and (feature_end <= flx_end) and (feature_start >= flx_start):
                    continue

# set up the features that will be added to the GTF

mCherry = ['V','mCherry','exon','1','6241','.','+','.',  ###first 8 elements, tab delimited
            'gene_id "mCherry"',   ### remaining elements, semicolon delimited
            ' gene_version "1"',
            ' transcript_id "mCherry"',
            ' transcript_version "1"',
            ' gene_name "mCherry"',
            ' gene_source "addgene"',
            ' gene_biotype "protein_coding"',
            ' transcript_name "mCherry"',
            ' transcript_source "custom"',
            ' transcript_biotype "protein_coding"',
            ' protein_id "mCherry"',
            ' tag "basic";']
mCherry_line_part1 = '\t'.join((mCherry[0:8]))
mCherry_line_part2 = ';'.join((mCherry[8:20]))
mCherry_final = '\t'.join((mCherry_line_part1,mCherry_line_part2))  ## there is one tab separation

nr1_flx_site = ['2','Grin1flx','exon',str(flx_start),str(flx_end),'.','-','.', 
            'gene_id "Grin1flx"',
            ' gene_version "1"',
            ' transcript_id "Grin1flx"',
            ' transcript_version "1"',
            ' gene_name "Grin1flx"',
            ' gene_source "addgene"',
            ' gene_biotype "protein_coding"',
            ' transcript_name "Grin1flx"',
            ' transcript_source "custom"',
            ' transcript_biotype "protein_coding"',
            ' protein_id "Grin1flx"',
            ' tag "basic";']

nr1_flx_site_line_part1 = '\t'.join((nr1_flx_site[0:8]))
nr1_flx_site_line_part2 = ';'.join((nr1_flx_site[8:20]))
nr1_flx = '\t'.join((nr1_flx_site_line_part1,nr1_flx_site_line_part2))  ## there is one tab separation

final_genes = '\n'.join((mCherry_final,nr1_flx))

# write the features to the new gtf file

with open(mod_gtf_path, "a") as myfile:
    myfile.write(final_genes)
    myfile.close()

In [5]:
# filter the gtf, removes unneccesary elements that are not needed to reduce overlap

filter_command = "cellranger mkgtf {} {} --attribute=gene_biotype:protein_coding \
                   --attribute=gene_biotype:lincRNA \
                   --attribute=gene_biotype:antisense \
                   --attribute=gene_biotype:IG_LV_gene \
                   --attribute=gene_biotype:IG_V_gene \
                   --attribute=gene_biotype:IG_V_pseudogene \
                   --attribute=gene_biotype:IG_D_gene \
                   --attribute=gene_biotype:IG_J_gene \
                   --attribute=gene_biotype:IG_J_pseudogene \
                   --attribute=gene_biotype:IG_C_gene \
                   --attribute=gene_biotype:IG_C_pseudogene \
                   --attribute=gene_biotype:TR_V_gene \
                   --attribute=gene_biotype:TR_V_pseudogene \
                   --attribute=gene_biotype:TR_D_gene \
                   --attribute=gene_biotype:TR_J_gene \
                   --attribute=gene_biotype:TR_J_pseudogene \
                   --attribute=gene_biotype:TR_C_gene".format(mod_gtf_path,filtered_gtf_path)
os.system(filter_command)

0

In [6]:
# construct the bed file

bed_command = "gtf2bed < {} > {}".format(filtered_gtf_path,bed_path)
os.system(bed_command)

0

In [8]:
# construct the mapping genome
os.chdir('/home_fmi/01/otoosean/home_faimsrv01/Sequel/opt_items/mcherry_nr1_genome_expand')
make_genome_command = "cellranger mkref --genome={} --fasta={} --genes={} --nthreads=30".format(genome_name,mod_fasta_path,filtered_gtf_path)
os.system(make_genome_command)


0